# statistical mean field of binned o2 data
- load intermediate binned data
- generate monthly climatology in 12 x Nlev x 180 x 360
- combine OSD and CTD

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import xarray as xr
import pandas as pd
import os

In [2]:
# data source identification
Nlev=47
# define standard depth
zstd=np.array([0.00e+00, 5.00e+00, 1.00e+01, 1.50e+01, 2.00e+01, 2.50e+01,
       3.00e+01, 3.50e+01, 4.00e+01, 4.50e+01, 5.00e+01, 5.50e+01,
       6.00e+01, 6.50e+01, 7.00e+01, 7.50e+01, 8.00e+01, 8.50e+01,
       9.00e+01, 9.50e+01, 1.00e+02, 1.25e+02, 1.50e+02, 1.75e+02,
       2.00e+02, 2.25e+02, 2.50e+02, 2.75e+02, 3.00e+02, 3.25e+02,
       3.50e+02, 3.75e+02, 4.00e+02, 4.25e+02, 4.50e+02, 4.75e+02,
       5.00e+02, 5.50e+02, 6.00e+02, 6.50e+02, 7.00e+02, 7.50e+02,
       8.00e+02, 8.50e+02, 9.00e+02, 9.50e+02, 1.00e+03, 1.05e+03,
       1.10e+03, 1.15e+03, 1.20e+03, 1.25e+03, 1.30e+03, 1.35e+03,
       1.40e+03, 1.45e+03, 1.50e+03, 1.55e+03, 1.60e+03, 1.65e+03,
       1.70e+03, 1.75e+03, 1.80e+03, 1.85e+03, 1.90e+03, 1.95e+03,
       2.00e+03, 2.10e+03, 2.20e+03, 2.30e+03, 2.40e+03, 2.50e+03,
       2.60e+03, 2.70e+03, 2.80e+03, 2.90e+03, 3.00e+03, 3.10e+03,
       3.20e+03, 3.30e+03, 3.40e+03, 3.50e+03, 3.60e+03, 3.70e+03,
       3.80e+03, 3.90e+03, 4.00e+03, 4.10e+03, 4.20e+03, 4.30e+03,
       4.40e+03, 4.50e+03, 4.60e+03, 4.70e+03, 4.80e+03, 4.90e+03,
       5.00e+03, 5.10e+03, 5.20e+03, 5.30e+03, 5.40e+03, 5.50e+03],
      dtype='float32')

### calculate the climatology of the Bottle data

In [ ]:
dirname='intermed_files'
# main loop to bin the profiles and save netCDF annually
YR=np.arange(1965,2022,1)
xc=np.arange(-180,180,1)+.5
yc=np.arange(-90,90,1)+.5
zc=zstd[:Nlev]
var='o2'
sumo2=np.zeros((12,Nlev,180,360))
dd=np.zeros((12,Nlev,180,360))
for yr in YR:
    print('=== working on '+str(yr)+' ===')
    fn=dirname+'/'+var+'_1x1bin_osd_'+str(yr)+'.nc'
    ds=xr.open_dataset(fn)
    o2loc=ds.o2.to_numpy()
    o2loc[np.isnan(o2loc)]=0
    nloc=ds.sample_count.to_numpy()
    nloc[np.isnan(nloc)]=0
    sumo2=sumo2+nloc*o2loc
    dd=dd+ds.sample_count.to_numpy()
# statistical mean
o2=sumo2/dd

In [ ]:
#zc=zstd[:Nlev]
time=np.arange('1980-01','1981-01',dtype='datetime64[M]')
da=xr.DataArray(data=o2,name='o_mn',dims=['time','depth','lat','lon'],\
               coords={'time':time,'depth':zc,'lat':yc,'lon':xc})
dd=xr.DataArray(data=dd,name='o_dd',dims=['time','depth','lat','lon'],\
               coords={'time':time,'depth':zc,'lat':yc,'lon':xc})
ds=da.to_dataset()
ds['o_dd']=dd
ds.to_netcdf('o2_clim_osd.nc')

In [ ]:
ds=xr.open_dataset('o2_clim_osd.nc')
ds.o_mn.mean('time').sel(depth=0).plot(vmax=400)

### calculate the climatology of CTD data

In [ ]:
## do the same for CTD data
var='o2'
YR=np.arange(1987,2021,1)
sumo2=np.zeros((12,Nlev,180,360))
dd=np.zeros((12,Nlev,180,360))
for yr in YR:
    print('=== working on '+str(yr)+' ===')
    fn=dirname+'/'+var+'_1x1bin_ctd_'+str(yr)+'.nc'
    ds=xr.open_dataset(fn)
    o2loc=ds.o2.to_numpy()
    o2loc[np.isnan(o2loc)]=0
    sumo2=sumo2+ds.sample_count.to_numpy()*o2loc
    dd=dd+ds.sample_count.to_numpy()
# statistical mean
o2=sumo2/dd

In [ ]:
time=np.arange('1980-01','1981-01',dtype='datetime64[M]')
da1=xr.DataArray(data=o2,name='o_mn',dims=['time','depth','lat','lon'],\
               coords={'time':time,'depth':zc,'lat':yc,'lon':xc})
dd1=xr.DataArray(data=dd,name='o_dd',dims=['time','depth','lat','lon'],\
               coords={'time':time,'depth':zc,'lat':yc,'lon':xc})
ds1=da1.to_dataset()
ds1['o_dd']=dd1
ds1.to_netcdf('o2_clim_ctd.nc')

In [ ]:
ds1=xr.open_dataset('o2_clim_ctd.nc')
ds1.o_mn.mean('time').sel(depth=0).plot(vmax=400)

In [ ]:
ds00=xr.open_dataset('o2_clim_osd.nc')
ds11=xr.open_dataset('o2_clim_ctd.nc')
ds11.o_dd.sum('time').sel(depth=0).plot(vmax=1)

### combine the two climatology into a single one

In [ ]:
# now combine two into a single one
#
sumo2=np.zeros((12,Nlev,180,360))
dd=np.zeros((12,Nlev,180,360))
#
o20=ds00.o_mn.to_numpy()
o20[np.isnan(o20)]=0
#
o21=ds11.o_mn.to_numpy()
o21[np.isnan(o21)]=0
#
sumo2=ds00.o_dd.to_numpy()*o20+ds11.o_dd.to_numpy()*o21
dd=ds00.o_dd.to_numpy()+ds11.o_dd.to_numpy()
o2=sumo2/dd
#
da2=xr.DataArray(data=o2,name='o_mn',dims=['time','depth','lat','lon'],\
               coords={'time':time,'depth':zc,'lat':yc,'lon':xc})
dd2=xr.DataArray(data=dd,name='o_dd',dims=['time','depth','lat','lon'],\
               coords={'time':time,'depth':zc,'lat':yc,'lon':xc})
ds2=da2.to_dataset()
ds2['o_dd']=dd2
ds2.to_netcdf('o2_clim_osdctd.nc')

In [ ]:
ds2=xr.open_dataset('o2_clim_osdctd.nc')
ds2.o_mn.mean('time').sel(depth=0).plot(robust=True)

In [ ]:
ds2.o_mn.sel(depth=0).plot(col='time',col_wrap=4)

In [ ]:
# pass 2 climatology uses 3 month moving window composite
o20=ds2.o_mn.to_numpy()
o2=np.concatenate([o20,o20,o20],axis=0)
o2rm=np.zeros(np.shape(o2))
for m in np.arange(1,35,1):
    d=np.empty((3,47,180,360))
    d[0,:,:,:]=o2[m-1,:,:,:]
    d[1,:,:,:]=o2[m,:,:,:]
    d[2,:,:,:]=o2[m+1,:,:,:]
    o2rm[m,:,:,:]=np.nanmean(d,axis=0)
o2rm1=o2rm[12:24,:,:,:]
da3=xr.DataArray(data=o2rm1,name='o_mn',dims=['time','depth','lat','lon'],\
               coords={'time':time,'depth':zc,'lat':yc,'lon':xc})
ds4=da3.to_dataset()
ds4.to_netcdf('o2_clim_osdctd_pass2.nc')

In [ ]:
ds4.o_mn.sel(depth=0).plot(col='time',col_wrap=4)